In [ ]:
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('english_cleaned_lyrics.csv')

In [ ]:
df.drop(columns='Unnamed: 0', inplace = True)
df.drop(columns='index', inplace = True)

In [ ]:
display(df)

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull().any(axis=1)]

In [2]:
pwd

'C:\\Users\\g_tro\\Algorithmes_supervisés'